In [1]:
import os
import phoenix
import phoenix as px
import pandas as pd
import openai
import re
import nest_asyncio
nest_asyncio.apply()

In [2]:
# 將 PHOENIX_COLLECTOR_ENDPOINT 指向本機實際 IP
#os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "http://10.234.14.175:6006" #學校
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "http://phoenix:6006" #家
client = px.Client()
dataset = client.get_dataset(
    name="only3-dataset"
    # ,version_id="RGF0YXNldFZlcnNpb246MTE="
)

/opt/conda/lib/python3.10/site-packages/phoenix/utilities/client.py:60: UserWarning: The Phoenix server (10.11.0) and client (10.12.0) versions are mismatched and may have compatibility issues.
  warnings.warn(


In [ ]:
#df = dataset.as_dataframe()
#print(df.columns)
#df["input"] = df["input"].apply(lambda x: x['question'])
#df["output"] = df["output"].apply(lambda x: x['respond'])
#df["metadata"] = df["metadata"].apply(lambda x: x['reference'])


In [3]:
export_records = [] #把資料存到資料表中

client = openai.OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_API_BASE")
)



async def explain(example, prompt_template: str, model: str) -> str:
#     now = datetime.now()
#     time_str = now.strftime("%Y-%m-%d %H:%M:%S")
# #     global task_result
# #     task_result = time_str
    
#     time.sleep(2)
    
#     return "[TASK] " + time_str

    formatted_prompt_template = prompt_template.format(input_message=example.input["question"],output_message=example.output["respond"],reference_message=example.metadata["reference"])
    
    formatted_prompt_template = formatted_prompt_template + """

# Example
```json
{
  "result": "事實",
  "explaination": "水在標準大氣壓下的沸點是攝氏100度。"
}
```

```json
{
  "result": "幻覺",
  "explaination": "回答內容描述水的沸點是華氏100度，但應該是攝氏100度。"
}
```

"""
    
    # print(formatted_prompt_template)

    res = client.chat.completions.create(
        model="o1",
        temperature=0,
        messages=[
            {"role": "user", "content": formatted_prompt_template},
        ],
    )
    assert res.choices
    content = res.choices[0].message.content
    
    return extract_brace_content(content)

In [4]:
import textwrap
from functools import partial

# template = """
# 你是一個判斷事實真偽的專家，請根據{reference_message}的內容，判斷{output_message}對於{input_message}的回答中，是否有與事實不符的地方。

# 以條列解釋你如何判斷為"事實"或"幻覺"
# """

template_json = """
# Role
你是一個判斷事實真偽的專家。

# Task
請根據{reference_message}的內容，判斷{output_message}對於{input_message}的回答中，是否有與事實不符的地方。

# Output
以JSON的型式，輸出以下資訊:
- result: "事實"或"幻覺"
- explaination: 你的解釋，以條列型式呈現，結合成一個字串。
"""

task = partial(explain, prompt_template=template_json, model="o1")
# print(dataset.examples)
test_example = dataset[0]
print(textwrap.fill(await task(test_example), width=100))

根據你提供的文本「艾菲爾鐵塔位於法國巴黎。它建於 1889 年，作為 1889 年世界博覽會的入口拱門。」，我可以判斷「艾菲爾鐵塔位於法國巴黎」是 **事實**。  以下是我的判斷依據：  *
**文本明確指出：** 文本的第一句話直接陳述「艾菲爾鐵塔位於法國巴黎」。 *   **廣為人知的事實：** 艾菲爾鐵塔是法國巴黎的著名地標，這是一個廣為人知的地理事實。 *
**沒有矛盾資訊：** 文本中沒有任何資訊與艾菲爾鐵塔位於巴黎相矛盾。  因此，根據文本內容，以及普遍的常識，我可以肯定地判斷艾菲爾鐵塔位於法國巴黎是 **事實**，而非幻覺。


In [5]:
from datetime import datetime

import json

def judge(metadata, output, input, reference, expected) -> str:
    try:
        # 嘗試解析 JSON 格式
        data = json.loads(output)
    except json.JSONDecodeError:
        # 若是單引號或非標準格式，轉換成可解析格式
        input_str = input_str.replace("'", '"')
        try:
            data = json.loads(input_str)
        except json.JSONDecodeError:
            return None  # 無法解析
    return data.get("result")

    # metadata, output, input, reference, expected.
    # now = datetime.now()
    # time_str = now.strftime("%Y-%m-%d %H:%M:%S")
    # return time_str
#     print({
#         "metadata": metadata,
#         "output": output,
#         "input": input,
#         "reference": reference,
#         "expected": expected,
#     })
    
#     return "[JUDGE]" + task_result
    # return output

    # =========================
    
#     input_message = input["question"]
#     output_message = expected["respond"]
#     reference_message = metadata["reference"]
#     # print(input_message)
#     # print(output_message)
#     # print(reference_message)
    
#     MY_CUSTOM_TEMPLATE = f'''

# [BEGIN DATA]
# [Question]: {input_message}
# [Response]: {output_message}
# [Reference]: {reference_message}
# [END DATA]

# # 角色
# 你是一個判斷事實真偽的專家，請根據[Reference]的內容，判斷[Response]對於[Question]的回答中，是否有與事實不符的地方。

# # 輸出格式
# "事實"或"幻覺"  

# '''
    
#     # print("==============================")
#     # print(MY_CUSTOM_TEMPLATE)
    
#     res = client.chat.completions.create(
#         model="o1",
#         temperature=0,
#         messages=[
#             {"role": "user", "content": MY_CUSTOM_TEMPLATE},
#         ]
#     )
#     content = res.choices[0].message.content
#     content = content.strip().lower()
#     return content

EVALUATORS = [judge]

In [6]:
from phoenix.experiments import run_experiment
experiment= run_experiment(
    dataset,
    task,
    experiment_name="explanation",
    experiment_metadata={"vendor": "openai", "model": "o1"},
    evaluators=EVALUATORS,
)

🧪 Experiment started.
📺 View dataset experiments: http://phoenix:6006/datasets/RGF0YXNldDoy/experiments
🔗 View this experiment: http://phoenix:6006/datasets/RGF0YXNldDoy/compare?experimentId=RXhwZXJpbWVudDox


running tasks |          | 0/3 (0.0%) | ⏳ 00:00<? | ?it/s

✅ Task runs completed.
🧠 Evaluation started.


running experiment evaluations |          | 0/3 (0.0%) | ⏳ 00:00<? | ?it/s

艾菲爾鐵塔位於哪裡？ 
根據你提供的內容「艾菲爾鐵塔位於法國巴黎。它建於 1889 年，作為 1889 年世界博覽會的入口拱門。」，以及你的問題「艾菲爾鐵塔位於法國巴黎。對於艾菲爾鐵塔位於哪裡？ 的回答中，是否有與事實不符的地方。」

我的判斷是：**事實**。

以下是我判斷的條列解釋：

*   **內容明確指出：** 文本直接且明確地聲稱「艾菲爾鐵塔位於法國巴黎」。
*   **普遍認知：** 艾菲爾鐵塔位於巴黎是廣為人知的地理事實，與歷史資料和實際情況相符。
*   **沒有矛盾資訊：** 文本中沒有任何資訊與艾菲爾鐵塔位於巴黎的陳述相矛盾。
*   **歷史背景佐證：** 文本提及艾菲爾鐵塔建於 1889 年世界博覽會，該博覽會確實於巴黎舉行，進一步佐證了艾菲爾鐵塔位於巴黎的說法。

因此，根據提供的文本和普遍認知，艾菲爾鐵塔位於法國巴黎的說法是**真實的**，並不存在與事實不符的地方。
艾菲爾鐵塔位於法國巴黎。它建於 1889 年，作為 1889 年世界博覽會的入口拱門。 
世界上最長的河流是什麼？ 
好的，我來分析你提供的資訊，並判斷其中是否有與事實不符的地方。

**判斷結果：** 部分正確，但有爭議。

**判斷過程：**

1.  **尼羅河向北流經非洲東北部，從布隆迪最遠的源頭流入地中海：** 這是大致正確的。尼羅河的源頭通常被認為位於非洲東部高原，最遠的源頭之一是位於布隆迪的卡蓋拉河。它確實向北流經東非、蘇丹和埃及，最終注入地中海。

2.  **尼羅河全長約 6,650 公里（4,132 英里）：** 這個長度是過去常用的數據，但現在存在爭議。近年來的測量和研究表明，亞馬遜河的長度可能超過尼羅河。

3.  **尼羅河是世界上最長的河流：** 這是一個有爭議的說法。長期以來，尼羅河被認為是世界上最長的河流。然而，近年來的研究表明，亞馬遜河可能更長。關於哪條河流更長，目前仍存在一些爭論，因為測量河流長度的定義和方法存在差異。

**總結：**

*   **事實：** 尼羅河流經非洲東北部，最終注入地中海。
*   **部分事實/有爭議：** 尼羅河全長約 6,650 公里（4,132 英里）。這個長度數據在過去是準確的，但現在可能已被亞馬遜河超越。
*   **幻覺/不準確：** 尼羅河絕對是世界上最長的河流。這個說法目前存在爭議，亞馬遜河

/opt/conda/lib/python3.10/site-packages/phoenix/utilities/client.py:60: UserWarning: The Phoenix server (10.11.0) and client (10.12.0) versions are mismatched and may have compatibility issues.
  warnings.warn(


人體有幾塊骨頭？ 
根據你提供的文本：「人體有206塊骨頭。這些骨骼提供結構、保護器官、固定肌肉並儲存鈣。」 以及你的問題「成年人的身體通常有 256 塊骨頭」，我可以判斷出後者為**幻覺**。

以下是我的判斷過程：

*   **文本陳述：** 文本明確指出人體有206塊骨頭。
*   **問題陳述：** 問題聲稱人體有256塊骨頭。
*   **比較：** 256與206不符。
*   **結論：** 由於問題中的骨骼數量與文本中提供的已知事實不符，因此可以判斷問題中的說法為**幻覺**。

因此，對於人體有幾塊骨頭的正確答案是 **206** 塊。
人體有206塊骨頭。這些骨骼提供結構、保護器官、固定肌肉並儲存鈣。 

🔗 View this experiment: http://phoenix:6006/datasets/RGF0YXNldDoy/compare?experimentId=RXhwZXJpbWVudDox

Experiment Summary (06/16/25 02:36 PM +0000)
--------------------------------------------
  evaluator  n  n_labels        top_2_labels
0     judge  3         3  {'事實': 2, '幻覺': 1}

Tasks Summary (06/16/25 02:36 PM +0000)
---------------------------------------
   n_examples  n_runs  n_errors
0           3       3         0
